In [1]:
from src.training import *
import warnings
warnings.filterwarnings("ignore", message="Unknown entity 'ee-infinity-loader'")

dims = (20,20)
rv = 4
datalist = prepare_dataset(dims=dims, repr_version=rv)
# datalist[0] = [d.get_matrix(dims, rv) for d in datalist[0]]
# datalist[2] = [d.get_matrix(dims, rv) for d in datalist[2]]

dataloader = DataLoader(
    AugmentedListDataset(*datalist),
    batch_size=32, 
    collate_fn=collate_numpy_matrices
)
train_dataloader, val_dataloader = split_dataloader(dataloader)
model = DeepQCNN(input_channels=8, output_channels=8).float()

trainer = QCNNTrainer(
        model=model,
        device='mps',
        learning_rate=0.001,
        log_dir='runs/qcnn',
    )

# Train model
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    num_epochs=100,
    save_path='models/qcnn_best.pth'
)

AttributeError: 'tuple' object has no attribute 'to'

In [ ]:
model = DeepQCNN(input_channels=8, output_channels=8)

for name, param in model.named_parameters():
    print(f"Parameter {name}: {param.dtype}")

Parameter conv1.weight: torch.float32
Parameter conv1.bias: torch.float32
Parameter conv2.weight: torch.float32
Parameter conv2.bias: torch.float32
Parameter conv3.weight: torch.float32
Parameter conv3.bias: torch.float32
Parameter dense1.weight: torch.float32
Parameter dense1.bias: torch.float32
Parameter dense2.weight: torch.float32
Parameter dense2.bias: torch.float32


: 

In [ ]:
from src.training import *
import warnings
warnings.filterwarnings("ignore", message="Unknown entity 'ee-infinity-loader'")

dims = (20,20)
rv = 4
datalist = prepare_dataset(dims=dims, repr_version=rv)
auggie = AugmentedListDataset(*datalist)

auggie

: 

In [ ]:
auggie[1000]

(array([[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0, 27,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        ...,
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],


: 

: 